# Spark Rasndom Forest

## classify colleges as Private or Public

In [3]:
#imports
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treecode').getOrCreate()

#data
data = spark.read.csv('../Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Tree_Methods/College.csv',inferSchema=True,header=True)
print(data.columns, '\n')
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Apps','Accept','Enroll','Top10perc','Top25perc','F_Undergrad','P_Undergrad','Outstate','Room_Board','Books','Personal','PhD','Terminal','S_F_Ratio','perc_alumni','Grad_Rate'],
                            outputCol="features")
output = assembler.transform(data)
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Private", outputCol="PrivateIndex")
output_fixed = indexer.fit(output).transform(output)

#split
final_data = output_fixed.select("features",'PrivateIndex')
train_data,test_data = final_data.randomSplit([0.7,0.3])

#model
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex',featuresCol='features')
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)
dtc_predictions = dtc_model.transform(test_data)
rfc_predictions = rfc_model.transform(test_data)
gbt_predictions = gbt_model.transform(test_data)

#evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="PrivateIndex", predictionCol="prediction", metricName="accuracy")
dtc_acc = acc_evaluator.evaluate(dtc_predictions)
rfc_acc = acc_evaluator.evaluate(rfc_predictions)
gbt_acc = acc_evaluator.evaluate(gbt_predictions)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbt_acc*100))

['School', 'Private', 'Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad', 'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD', 'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'] 

A single decision tree had an accuracy of: 92.73%
A random forest ensemble had an accuracy of: 95.91%
A ensemble using GBT had an accuracy of: 90.91%
